In [ ]:
!pip install rdflib

In [3]:
from rdflib import Graph, Namespace, RDF, RDFS, XSD

## Simple example of code

In [ ]:
# Create a Graph
g = Graph()

# Parse in an RDF file hosted on the Internet
g.parse("http://www.w3.org/People/Berners-Lee/card")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")
# Prints: Graph g has 86 statements.

# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="turtle"))

## Start to define the TBox

In [5]:
g = Graph()
pub = Namespace("http://www.example.org/publication#")
g.bind("pub", pub)

Here we will define the class and the subclass of the schema

In [9]:
# Class
g.add((pub.Scientist, RDF.type, RDFS.Class))
g.add((pub.Publication, RDF.type, RDFS.Class))
g.add((pub.Review, RDF.type, RDFS.Class))
g.add((pub.Approval, RDF.type, RDFS.Class))
g.add((pub.Topic, RDF.type, RDFS.Class))
g.add((pub.Community, RDF.type, RDFS.Class))
g.add((pub.Workshop, RDF.type, RDFS.Class))
g.add((pub.Journal, RDF.type, RDFS.Class))
g.add((pub.Event, RDF.type, RDFS.Class))
g.add((pub.Organization, RDF.type, RDFS.Class))
#Subclass
#Scientists
g.add((pub.Guru, RDF.type, RDFS.Class))
g.add((pub.Guru, RDFS.subClassOf, pub.Scientist))
g.add((pub.Potential_good, RDF.type, RDFS.Class))
g.add((pub.Potential_good, RDFS.subClassOf, pub.Scientist))
g.add((pub.Reviewer, RDF.type, RDFS.Class))
g.add((pub.Reviewer, RDFS.subClassOf, pub.Scientist))
#Publications
g.add((pub.TopPaper, RDF.type, RDFS.Class))
g.add((pub.TopPaper, RDFS.subClassOf, pub.Publication))
#Events
g.add((pub.Conference, RDF.type, RDFS.Class))
g.add((pub.Conference, RDFS.subClassOf, pub.Event))
g.add((pub.Workshop, RDF.type, RDFS.Class))
g.add((pub.Workshop, RDFS.subClassOf, pub.Event))
g.add((pub.Journal, RDF.type, RDFS.Class))
g.add((pub.Journal, RDFS.subClassOf, pub.Event))

<Graph identifier=Nbf180b6463264685a1209039638afd05 (<class 'rdflib.graph.Graph'>)>

In [10]:
#Scientist
g.add((pub.writes, RDF.type, RDF.Property))
g.add((pub.writes, RDFS.domain, pub.Scientist))
g.add((pub.writes, RDFS.range, pub.Publication))
g.add((pub.reviews, RDF.type, RDF.Property))
g.add((pub.reviews, RDFS.domain, pub.Reviewer))
g.add((pub.reviews, RDFS.range, pub.Publication))
g.add((pub.affiliations, RDF.type, RDF.Property))
g.add((pub.affiliations, RDFS.domain, pub.Scientist))
g.add((pub.affiliations, RDFS.range, pub.Organization))
g.add((pub.sends, RDF.type, RDF.Property))
g.add((pub.sends, RDFS.domain, pub.Scientist))
g.add((pub.sends, RDFS.range, pub.Review))

#Publication
g.add((pub.cites, RDF.type, RDF.Property))
g.add((pub.cites, RDFS.domain, pub.Publication))
g.add((pub.cites, RDFS.range, pub.Publication))
g.add((pub.discuss, RDF.type, RDF.Property))
g.add((pub.discuss, RDFS.domain, pub.Publication))
g.add((pub.discuss, RDFS.range, pub.Topic))
g.add((pub.title, RDF.type, RDF.Property))
g.add((pub.title, RDFS.domain, pub.Publication))
g.add((pub.title, RDFS.range, XSD.string))
g.add((pub.abstract, RDF.type, RDF.Property))
g.add((pub.abstract, RDFS.domain, pub.Publication))
g.add((pub.abstract, RDFS.range, XSD.string))

g.add((pub.containsConference, RDF.type, RDF.Property))
g.add((pub.containsConference, RDFS.domain, pub.Conference))
g.add((pub.containsConference, RDFS.range, pub.Publication))

g.add((pub.cityConference, RDF.type, RDF.Property))
g.add((pub.cityConference, RDFS.domain, pub.Conference))
g.add((pub.cityConference, RDFS.range, XSD.string))

g.add((pub.yearConference, RDF.type, RDF.Property))
g.add((pub.yearConference, RDFS.domain, pub.Conference))
g.add((pub.yearConference, RDFS.range, XSD.string))

g.add((pub.proceedingIDConference, RDF.type, RDF.Property))
g.add((pub.proceedingIDConference, RDFS.domain, pub.Conference))
g.add((pub.proceedingIDConference, RDFS.range, XSD.string))

g.add((pub.containsWorkshop, RDF.type, RDF.Property))
g.add((pub.containsWorkshop, RDFS.domain, pub.Workshop))
g.add((pub.containsWorkshop, RDFS.range, pub.Publication))

g.add((pub.cityWorkshop, RDF.type, RDF.Property))
g.add((pub.cityWorkshop, RDFS.domain, pub.Workshop))
g.add((pub.cityWorkshop, RDFS.range, XSD.string))

g.add((pub.yearWorkshop, RDF.type, RDF.Property))
g.add((pub.yearWorkshop, RDFS.domain, pub.Workshop))
g.add((pub.yearWorkshop, RDFS.range, XSD.string))

g.add((pub.proceedingIDWorkshop, RDF.type, RDF.Property))
g.add((pub.proceedingIDWorkshop, RDFS.domain, pub.Workshop))
g.add((pub.proceedingIDWorkshop, RDFS.range, XSD.string))

g.add((pub.containsJournal, RDF.type, RDF.Property))
g.add((pub.containsJournal, RDFS.domain, pub.Journal))
g.add((pub.containsJournal, RDFS.range, pub.Publication))

g.add((pub.volume, RDF.type, RDF.Property))
g.add((pub.volume, RDFS.domain, pub.Journal))
g.add((pub.volume, RDFS.range, XSD.string))

g.add((pub.yearJournal, RDF.type, RDF.Property))
g.add((pub.yearJournal, RDFS.domain, pub.Journal))
g.add((pub.yearJournal, RDFS.range, XSD.string))


g.add((pub.nameCommunity, RDF.type, RDF.Property))
g.add((pub.nameCommunity, RDFS.domain, pub.Community))
g.add((pub.nameCommunity, RDFS.range, XSD.string))

g.add((pub.related, RDF.type, RDF.Property))
g.add((pub.related, RDFS.domain, pub.Community))
g.add((pub.related, RDFS.range, pub.Event))

g.add((pub.publishes, RDF.type, RDF.Property))
g.add((pub.publishes, RDFS.domain, pub.Approval))
g.add((pub.publishes, RDFS.range, pub.Publication))

g.add((pub.considers, RDF.type, RDF.Property))
g.add((pub.considers, RDFS.domain, pub.Approval))
g.add((pub.considers, RDFS.range, pub.Review))

g.add((pub.totalReviewCount, RDF.type, RDF.Property))
g.add((pub.totalReviewCount, RDFS.domain, pub.Approval))
g.add((pub.totalReviewCount, RDFS.range, XSD.integer))

g.add((pub.approvalCount, RDF.type, RDF.Property))
g.add((pub.approvalCount, RDFS.domain, pub.Approval))
g.add((pub.approvalCount, RDFS.range, XSD.integer))

g.add((pub.status, RDF.type, RDF.Property))
g.add((pub.status, RDFS.domain, pub.Approval))
g.add((pub.status, RDFS.range, XSD.string))

g.add((pub.description, RDF.type, RDF.Property))
g.add((pub.description, RDFS.domain, pub.Review))
g.add((pub.description, RDFS.range, XSD.string))

g.add((pub.decision, RDF.type, RDF.Property))
g.add((pub.decision, RDFS.domain, pub.Review))
g.add((pub.decision, RDFS.range, XSD.string))

g.add((pub.nameOrganization, RDF.type, RDF.Property))
g.add((pub.nameOrganization, RDFS.domain, pub.Organization))
g.add((pub.nameOrganization, RDFS.range, XSD.string))

g.add((pub.type, RDF.type, RDF.Property))
g.add((pub.type, RDFS.domain, pub.Organization))
g.add((pub.type, RDFS.range, XSD.string))

<Graph identifier=Nbf180b6463264685a1209039638afd05 (<class 'rdflib.graph.Graph'>)>

## Save in the file rdfs

In [11]:
g.serialize("tbox_lab.ttl", format="turtle")

<Graph identifier=Nbf180b6463264685a1209039638afd05 (<class 'rdflib.graph.Graph'>)>